In [81]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam

# Convolution with Batch Norm & ReLU

In [110]:
class ConvBNReLU(Model):   
    def __init__(self, filters, kernel_size, strides=1):
        super().__init__()
        if(kernel_size == 1):
            padding = 0
        else:
            padding = 1        
        # Conv2D
        self.addPadding = ZeroPadding2D(padding)
        self.Conv = Conv2D(filters, kernel_size, strides)
        # Batch Normalization
        self.BN = BatchNormalization()
        # Activation Function
        self.AF = Activation("relu")
    def call(self, x):
        x = self.addPadding(x)
        x = self.Conv(x)
        x = self.BN(x)
        return self.AF(x)

# Depthwise Separable Convolution

In [108]:
class depthwise_separable(Model):
    def __init__(self, filters, kernel_size):
        super().__init__()
        #add padding
        self.addPadding = ZeroPadding2D(padding=1)
        # Depthwise Convolution 3x3
        self.DWSeparableConv = SeparableConv2D(filters, kernel_size)
        # Batch Normalization
        self.BN = BatchNormalization()
        # Activation Function
        self.AF = Activation("relu")
    def call(self, x):
        x = self.addPadding(x)
        x = self.DWSeparableConv(x)
        x = self.BN(x)
        return self.AF(x)

# Separable Squeeze-Expand Block

In [52]:
class SSEModel(Model):
    def __init__(self, squeeze, expand):
        super().__init__()
        #squeeze
        self.pointwise1 = ConvBNReLU(squeeze, 1)
        #expand
        self.pointwise2 = ConvBNReLU(expand, 1)
        self.depthwise = depthwise_separable(expand, 3)
    def call(self, x):
        x = self.pointwise1(x)
        y = self.pointwise2(x)
        z = self.depthwise(x)
        return tf.keras.layers.Concatenate()([y, z])

# Slim Module

In [107]:
class SlimModule(Model):
    def __init__(self, squeeze, expand, dws, filtersNo):
        super().__init__()
        #SSE block 1
        self.SSE1 = SSEModel(squeeze, expand)
        #skip connection
        self.SkipConnection = 0
        #Dense
        self.dns = Dense(filtersNo, activation='relu')
        #SSE block 2
        self.SSE2 = SSEModel(squeeze, expand)
        
        #depthwise separable conv missing?
        self.depthwise = depthwise_separable(dws, 3)
    def call(self, x):
        s1 = self.SSE1(x)
        s1 = self.dns(s1)
        SkipConnection = Add()([x, s1])
        s2 = self.SSE2(SkipConnection)
        return self.depthwise(s2)

# SlimNet - CNN

In [157]:
slim = Sequential()
slim.add(Input(shape=(256, 256, 3)))
slim.add(Conv2D(96, (7, 7), (2, 2), padding='same'))
slim.add(Activation('relu'))

#Slim module 1
slim.add(SlimModule(16, 64, 48, 96))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))

#Slim module 2
slim.add(SlimModule(32, 128, 96, 48))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))

#Slim module 3
slim.add(SlimModule(48, 192, 144, 96))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))

#Slim module 4
slim.add(SlimModule(64, 256, 192, 144))
slim.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))

slim.add(GlobalAveragePooling2D())

slim.add(Dense(1, activation='sigmoid'))


opt = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

slim.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

print(slim.summary())
        

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_931 (Conv2D)          (None, 128, 128, 96)      14208     
_________________________________________________________________
activation_1435 (Activation) (None, 128, 128, 96)      0         
_________________________________________________________________
slim_module_168 (SlimModule) (None, 128, 128, 48)      28816     
_________________________________________________________________
max_pooling2d_168 (MaxPoolin (None, 63, 63, 48)        0         
_________________________________________________________________
slim_module_169 (SlimModule) (None, 63, 63, 96)        62608     
_________________________________________________________________
max_pooling2d_169 (MaxPoolin (None, 31, 31, 96)        0         
_________________________________________________________________
slim_module_170 (SlimModule) (None, 31, 31, 144)     

# Check for Data

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator()
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('output/train/', class_mode='binary')
val_it = datagen.flow_from_directory('output/val/', class_mode='binary')
test_it = datagen.flow_from_directory('output/test/', class_mode='binary')
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 141819 images belonging to 1 classes.
Found 40519 images belonging to 1 classes.
Found 20261 images belonging to 1 classes.
Batch shape=(32, 256, 256, 3), min=0.000, max=255.000


# Train & Save model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("output/savedModel",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]


nb_train_samples = 20000
nb_validation_samples = 4000
epochs = 1
batch_size = 64
 
history = slim.fit(
    train_it,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = val_it,
    validation_steps = nb_validation_samples // batch_size)

slim.save("SlimNet_Model_opt-adam.h5")


 31/312 [=>............................] - ETA: 39:36 - loss: 0.1366 - accuracy: 0.9878